### Plot resampled variables for lrauv vehicles against aligned data 
To run this Notebook follow instructions at https://github.com/mbari-org/auv-python.

Processed files must be available locally. Execute `uv run src/data/process_lrauv.py` with the `--no_cleanup` option to create them,e.g.:
```
src/data/process_lrauv.py -v --log_file pontus/missionlogs/2024/20240715_20240725/20240723T023501/202407230235_202407232319.nc4 --no_cleanup
```

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src/data'));
if module_path not in sys.path:
    sys.path.append(module_path)
import xarray as xr
import hvplot.pandas
import hvplot.xarray
import holoviews as hv
import ipywidgets as widgets
import pandas as pd
from pathlib import Path
import netCDF4 as nc4
import logging
from nc42netcdfs import BASE_LRAUV_PATH

# Configure logging
logging.basicConfig(
    level=logging.INFO,  # Change to logging.DEBUG for more detailed output
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

# Enable bokeh extension for hvplot
hv.extension('bokeh')

# Get time coordinate for each variable by introspection
def get_time_coord(var):
    """Get the time coordinate name for a variable.
    
    Args:
        var: Either an xarray.DataArray or netCDF4.Variable
        
    Returns:
        str: Name of the time coordinate/dimension
    """
    # Check if it's an xarray DataArray (has .dims attribute)
    if hasattr(var, 'dims'):
        # xarray DataArray
        time_dims = [dim for dim in var.dims if 'time' in dim.lower()]
        return time_dims[0] if time_dims else var.dims[0]
    elif hasattr(var, 'dimensions'):
        # netCDF4 Variable
        time_dims = [dim for dim in var.dimensions if 'time' in dim.lower()]
        return time_dims[0] if time_dims else var.dimensions[0]
    else:
        raise TypeError(f"Unsupported variable type: {type(var)}")

# Pick the auv_name
auv_name = widgets.Dropdown(
    options=[f for f in sorted(os.listdir(BASE_LRAUV_PATH)) if f != ".DS_Store"],
    description='auv_name:',
    disabled=False,
)
display(auv_name)

In [ ]:
# Pick the log file from the selected LRAUV directory
# Pattern: {lrauv_name}/missionlogs/{year}/{date_range}/{mission_start}/{log_file}.nc4

lrauv_name = auv_name.value
log_files = sorted(Path(BASE_LRAUV_PATH).glob(f"{lrauv_name}/missionlogs/*/*/*/*[0-9].nc4"))
log_file_options = [str(f.relative_to(BASE_LRAUV_PATH)) for f in log_files]

log_file_picker = widgets.Select(
    options=log_file_options,
    description='Log File:',
    disabled=False,
    rows=15,
    layout=widgets.Layout(width='800px')
)
display(log_file_picker)

In [ ]:
# Load aligned and resampled files for comparison
log_file = log_file_picker.value
log_path = Path(BASE_LRAUV_PATH) / log_file
log_stem = log_path.stem
log_dir = log_path.parent

# Read the aligned and resampled netCDF files
aligned_file = log_file.replace('.nc4', '_align.nc4')
resampled_file = log_file.replace('.nc4', '_1S.nc')

aligned_ds = xr.open_dataset(os.path.join(BASE_LRAUV_PATH, aligned_file))
resampled_ds = xr.open_dataset(os.path.join(BASE_LRAUV_PATH, resampled_file))

logger.debug(f"Aligned file: {aligned_file}")
logger.debug(f"Resampled file: {resampled_file}")

# Show dataset details only in DEBUG mode
if logger.isEnabledFor(logging.DEBUG):
    logger.debug("\nAligned file:")
    display(aligned_ds)
    logger.debug("\nResampled file:")
    display(resampled_ds)

def plot_aligned_vs_resampled(variable_name):
    """Plot a variable from both aligned and resampled files overlaid.
    
    Args:
        variable_name: Name of the variable from resampled dataset
        
    Returns:
        Overlay plot of aligned and resampled data, or just resampled if variable not in aligned
    """
    logger.debug(f"Processing variable: {variable_name}")
    
    # Get resampled time coordinate and create resampled plot
    resampled_time_coord = get_time_coord(resampled_ds[variable_name])
    
    # Get units for ylabel
    try:
        units = resampled_ds[variable_name].attrs.get('units', '')
    except (AttributeError, KeyError):
        units = ''
    
    # Create ylabel with format "name (units)"
    ylabel = f"{variable_name} ({units})" if units else variable_name
    
    # Create resampled plot
    resampled_plot = resampled_ds[variable_name].hvplot.line(
        x=resampled_time_coord, 
        label='Resampled', 
        color='#DC3545',  # Red
        width=900, 
        height=400, 
        ylabel=ylabel, 
        title=log_file, 
        alpha=0.7
    )
    
    # Check if variable exists in aligned dataset
    if variable_name not in aligned_ds:
        logger.warning(f"Variable {variable_name} not found in aligned dataset - plotting resampled only")
        return resampled_plot
    
    # Get aligned time coordinate and create aligned plot
    aligned_time_coord = get_time_coord(aligned_ds[variable_name])
    logger.debug(f"Time coords - Aligned: {aligned_time_coord}, Resampled: {resampled_time_coord}")
    
    # Create aligned plot
    aligned_plot = aligned_ds[variable_name].hvplot.line(
        x=aligned_time_coord, 
        label='Aligned', 
        color='#28A745',  # Green
        width=900, 
        height=400, 
        ylabel=ylabel, 
        title=log_file, 
        alpha=0.7
    )
    
    logger.debug("Plots created successfully")
    
    # Overlay the plots on the same axes (order determines legend order: Aligned, Resampled)
    return aligned_plot * resampled_plot

# Select multiple variables to plot from the resampled dataset
variable_picker = widgets.SelectMultiple(
    options=sorted([var for var in resampled_ds.data_vars]),
    description='Variables:',
    disabled=False,
    rows=15,
    layout=widgets.Layout(width='800px')
)
display(variable_picker)

In [ ]:
# Plot all selected variables with linked axes
plots = []
for variable_name in variable_picker.value:
    plot = plot_aligned_vs_resampled(variable_name)
    if plot is not None:
        plots.append(plot)
display(hv.Layout(plots).cols(1).opts(shared_axes=True))